# Домашнее задание

Промптинг языковых моделей

**Цель:**

В этом задании вы будете работать с предобученными крупными языковыми моделями (LLM), создавать промпты и применять модели для решения NLP-задач с помощью промптинга.

**Описание/Пошаговая инструкция выполнения домашнего задания:**


1. Возьмите данные для тестирования https://github.com/RussianNLP/RuCoLA/blob/main/data/in_domain_dev.csv

2. Возьмите модель Qwen/Qwen2.5-1.5B-Instruct (https://huggingface.co/Qwen/Qwen2.5-1.5B-Instruct), придумайте промпт для решения данной задачи (пока без system promt) и протестируйте ее на данной задаче.

**Замечание:** не забудьте написать функцию постобработки ответов, которая по генерации модели будет сопоставлять лейбл.

3. Поэкспериментируйте с промптами:

    - на русском и на английском языке,
    - с разными формулировками и структурами.

Оцените, как эти изменения влияют на качество решения задачи.

4. Добавьте System Prompt и проведите аналогичный эксперимент, варьируя System Propmt

5. Проведите эксперимент аналогичный п.2 с моделью Qwen/Qwen2.5-1.5B (https://huggingface.co/Qwen/Qwen2.5-1.5B) или Qwen/Qwen2.5-7B

6. Сравните результаты экспериментов из пп.2-4, сделайте выводы о влиянии промпта и тому, приведите ваши соображения о том, достаточен ли размер модели для решения данной задачи с хорошим качеством.


In [4]:
import gc
import random
from functools import partial
from tqdm.auto import tqdm, trange

import numpy as np
import pandas as pd
pd.options.display.max_colwidth = 300

import torch
from torch.optim import Adam
from torch.utils.data import DataLoader

import accelerate

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
from transformers import pipeline, DataCollatorWithPadding
from datasets import Dataset, DatasetDict, load_metric
from razdel import tokenize

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

import warnings
warnings.filterwarnings("ignore")

## 1. Подготовка набора данных

Возьмите данные для тестирования https://github.com/RussianNLP/RuCoLA/blob/main/data/in_domain_dev.csv

In [7]:
#df_train = pd.read_csv('data/in_domain_train.csv', index_col=0)
df_test = pd.read_csv('data/in_domain_dev.csv', index_col=0)

In [8]:
df_test

,sentence,acceptable,error_type,detailed_source
id,,,,
0,Иван вчера не позвонил.,1,0,Paducheva2013
1,"У многих туристов, кто посещают Кемер весной, есть шанс застать снег на вершине горы Тахталы и даже сочетать пляжный отдых с горнолыжным.",0,Syntax,USE8
2,"Лесные запахи набегали волнами; в них смешалось дыхание можжевельника, вереска, брусники.",1,0,USE5
3,Вчера президент имел неофициальную беседу с английским послом.,1,0,Seliverstova
4,Коллега так и не признал вину за катастрофу перед коллективом.,1,0,Testelets
...,...,...,...,...
978,Мысли отказываются остановиться на всяком предмете.,0,Semantics,Paducheva2013
979,"Не должно быть подозрений, что судью привлекают только потому, что он высказал свое критическое отношение к каким бы то ни было действиям самой государственной власти.",0,Semantics,Paducheva2013
980,"Хорошо, что он купил что-нибудь.",0,Semantics,Rusgram


In [9]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 983 entries, 0 to 982
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   sentence         983 non-null    object
 1   acceptable       983 non-null    int64 
 2   error_type       983 non-null    object
 3   detailed_source  983 non-null    object
dtypes: int64(1), object(3)
memory usage: 38.4+ KB


In [10]:
df_test["acceptable"].nunique()

2

In [11]:
df_test["acceptable"].unique()

array([1, 0])

In [12]:
df_test.groupby(["acceptable"]).count()

,sentence,error_type,detailed_source
acceptable,,,
0,250,250,250
1,733,733,733


In [13]:
df_test["acceptable"].value_counts()

acceptable
1    733
0    250
Name: count, dtype: int64

In [14]:
baseline = 733/(733+250)
baseline

0.745676500508647

In [15]:
df_test["error_type"].nunique()

4

In [16]:
df_test["error_type"].unique()

array(['0', 'Syntax', 'Semantics', 'Morphology'], dtype=object)

In [17]:
ind = random.randint(0, df_test.shape[0]-1)
ind

340

In [18]:
df_test.sentence[ind]

'Если бы не низкие потолки, они бросились бы качать Отто.'

## 1.1. Подготовим датасет для работы с моделью

In [20]:
#train_ds = Dataset.from_dict({'text':df_train.sentence, 'label':df_train.acceptable}, split='train')
#train_ds

In [21]:
#train_ds['text'][0]

In [22]:
test_ds = Dataset.from_dict({'text':df_test.sentence, 'label':df_test.acceptable}, split='test')
test_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 983
})

In [23]:
test_ds['text'][-1]

'На Марсе есть какие-либо (какие бы то ни было) разумные обитатели.'

In [24]:
test_ds[0]['text'], test_ds[0]['label']

('Иван вчера не позвонил.', 1)

In [25]:
labels_true = test_ds['label']

In [26]:
len(labels_true)

983

## 2. Загрузим модель Qwen2.5-1.5B-Instruct

In [28]:
base_model = 'Qwen/Qwen2.5-1.5B-Instruct'

In [29]:
tokenizer = AutoTokenizer.from_pretrained(base_model)

In [30]:
type(tokenizer)

transformers.models.qwen2.tokenization_qwen2_fast.Qwen2TokenizerFast

In [31]:
model = AutoModelForCausalLM.from_pretrained(base_model)
model

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [33]:
model.to(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [34]:
context =  'Лесные запахи набегали волнами; в них смешалось дыхание можжевельника, вереска, брусники.' #"На Марсе есть какие-либо (какие бы то ни было) разумные обитатели."
prompt = 'Корректно ли предыдущее выражение ?'

In [35]:
messages = [
    {"role": "user", "content": context},
    {"role": "user", "content": prompt},

]
messages

[{'role': 'user',
  'content': 'Лесные запахи набегали волнами; в них смешалось дыхание можжевельника, вереска, брусники.'},
 {'role': 'user', 'content': 'Корректно ли предыдущее выражение ?'}]

In [36]:
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

In [37]:
inputs

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
            553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
             13, 151645,    198, 151644,    872,    198,  63914,  22484,  42965,
          29789, 126202,   1802, 140202,  24671,  47255, 141724,  49707,     26,
           5805, 129690, 126762,  45967,  15952,  98720,   7796,  62133,  26619,
          44483,  16964,  32642,  25428,  87496,     11,  63262,  22484,  13132,
             11,  14062, 129089, 129309,     13, 151645,    198, 151644,    872,
            198,  26338,   9062, 141251,  13685,  58095,  56825,   4552,   6949,
          42193,  49406, 143587,  17175,    937, 151645,    198, 151644,  77091,
            198]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [38]:
inputs["input_ids"].shape

torch.Size([1, 82])

In [39]:
outputs = model.generate(**inputs, max_new_tokens=4)

In [40]:
outputs[0]

tensor([151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
           553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
            13, 151645,    198, 151644,    872,    198,  63914,  22484,  42965,
         29789, 126202,   1802, 140202,  24671,  47255, 141724,  49707,     26,
          5805, 129690, 126762,  45967,  15952,  98720,   7796,  62133,  26619,
         44483,  16964,  32642,  25428,  87496,     11,  63262,  22484,  13132,
            11,  14062, 129089, 129309,     13, 151645,    198, 151644,    872,
           198,  26338,   9062, 141251,  13685,  58095,  56825,   4552,   6949,
         42193,  49406, 143587,  17175,    937, 151645,    198, 151644,  77091,
           198,  24110,   1478,     11,  56825], device='cuda:0')

In [41]:
answer = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:])
answer

'Да, пред'

In [42]:
1 if 'да' in answer.lower() else 0

1

Напишем функию для получения предсказанной метки

In [44]:
def get_label(context, prompt, sys_prompt=None, max_new_tokens=4):
    # preparing message template
    if sys_prompt:
        messages = [
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": context},
            {"role": "user", "content": prompt},
            ]
    else:
        messages = [
            {"role": "user", "content": context},
            {"role": "user", "content": prompt},
            ]
    # getting input for the model
    inputs = tokenizer.apply_chat_template(
    	messages,
    	add_generation_prompt=True,
    	tokenize=True,
    	return_dict=True,
    	return_tensors="pt",
    ).to(model.device)
    # generating output
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    # decoding answer
    answer = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:])
    label = 1 if 'да' in answer.lower() else 0
    print(answer, label)
    # returning predicted label
    return label   

In [45]:
# context = "На Марсе есть какие-либо (какие бы то ни было) разумные обитатели."
context = 'Лесные запахи набегали волнами; в них смешалось дыхание можжевельника, вереска, брусники.'
prompt = 'Корректно ли предыдущее выражение ? Отвечай только Да или Нет.'
print(get_label(context, prompt))

Нет<|im_end|> 0
0


Напишем цикл пакетной обработки датасета и сохраним предсказанные метки в массив

In [47]:
prompt = 'Корректно ли предыдущее выражение ?'
pred_labels = []
for item in tqdm(test_ds):
    #print(item['text'], item['label'])
    pred_labels.append(get_label(item['text'], prompt))

  0%|          | 0/983 [00:00<?, ?it/s]

Предложение " 0
Да, пред 1
Да, выраж 1
Предложение " 0
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Нет, это 0
Да, пред 1
Да, выраж 1
Да, пред 1
Да, выраж 1
Нет, это 0
Да, выраж 1
Да, пред 1
Да, выраж 1
Предложение " 0
Предложение " 0
Ваше пред 0
Да, пред 1
Да, выраж 1
Да, выраж 1
Да, выраж 1
Да, пред 1
Да, выраж 1
Да, выраж 1
Да, выраж 1
"В эту минут 0
Ваше пред 0
Да, пред 1
Предложение " 0
Да, пред 1
Нет, пред 0
Предложение " 0
Ваше пред 0
Да, пред 1
Предложение " 0
Да, пред 1
Да, выраж 1
Да, пред 1
Да, пред 1
Да, выраж 1
Да, выраж 1
Да, пред 1
Предложение " 0
Да, пред 1
Да, пред 1
Да, пред 1
Ваше пред 0
Да, пред 1
Да, пред 1
Да, выраж 1
Да, пред 1
Ваше пред 0
Да, пред 1
Да, пред 1
Нет, это 0
Да, выраж 1
Слова " 0
Да, пред 1
Да, пред 1
Да, пред 1
Предыд 0
"Всю 0
Да, выраж 1
Да, пред 1
Да, выраж 1
"Впереди 0
Выражение " 0
Да, пред 1
Предложение " 0
Да, пред 1
Предыд 0
Да, выраж 1
Да, выраж 1
Да, пред 1
Нет, это 0
Да, пред 1
Да, пред 1
Да, пред 1
Нет, пред 0
Нет, выраж 0
Ваше пред 0
Да,

In [48]:
sum([lt == lp for lt, lp in zip(labels_true, pred_labels)])/len(labels_true)

0.5544252288911495

In [49]:
accuracy_score(labels_true, pred_labels)

0.5544252288911495

## 3. Эксперименты с промптами

- на русском и на английском языке,
- с разными формулировками и структурами.


In [51]:
# Dictionary to store model results
results = {}

In [52]:
# Let's define prompt's constants
BASE_RU_PROMPT = 'Предыдущее предложение корректно ?'
BASE_EN_PROMPT = 'Is the previous sentence correct? Response translate to russian language.'
YESNO_RU_PROMPT = 'Предыдущее предложение корректно ? Отвечай только Да или Нет.'
YESNO_EN_PROMPT = 'Is the previous sentence correct? Answer only Yes or No. Response translate to russian language.'

**Напишем функцию для оценки результатов модели**

In [54]:
def get_model_results(prompt, y_true, sys_prompt=None):
    pred_labels = []
    for item in tqdm(test_ds):
        #print(item['text'], item['label'])
        pred_labels.append(get_label(item['text'], prompt, sys_prompt))
    #acc = sum([lt == lp for lt, lp in zip(labels_true, pred_labels)])/len(labels_true)
    acc = accuracy_score(y_true, pred_labels)
    pr = precision_score(y_true, pred_labels)
    rc = recall_score(y_true, pred_labels)
    f1 = f1_score(y_true, pred_labels)
    return [acc, pr, rc, f1]

### 3.1 Base ru prompt

In [56]:
prompt = BASE_RU_PROMPT
prompt

'Предыдущее предложение корректно ?'

In [57]:
results[model.name_or_path.split('/')[1] + ' / ru base prompt'] = get_model_results(prompt, labels_true)

  0%|          | 0/983 [00:00<?, ?it/s]

Да, пред 1
Да, пред 1
Ваше пред 0
Да, пред 1
Ваше пред 0
Ваше пред 0
Ваше пред 0
Да, пред 1
Нет, пред 0
Да, пред 1
Нет, пред 0
Нет, пред 0
Да, пред 1
Ваше пред 0
Извините 0
"Дело приня 0
Да, пред 1
Ваше пред 0
Ваша ф 0
Да, пред 1
Да, пред 1
Да, пред 1
Нет, пред 0
Да, пред 1
Да, пред 1
Ваше пред 0
Да, пред 1
Ваша ф 0
Извините 0
Да, пред 1
Нет, пред 0
Нет, пред 0
Да, пред 1
Ваше пред 0
Да, пред 1
Да, пред 1
Нет, пред 0
Да, пред 1
Ваше пред 0
Да, пред 1
Ваше пред 0
Да, пред 1
Да, пред 1
Ваше пред 0
Да, пред 1
Да, пред 1
Да, пред 1
Нет, пред 0
Нет, пред 0
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Ваше пред 0
Да, пред 1
Да, пред 1
Нет, пред 0
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Ваше пред 0
Нет, пред 0
Да, пред 1
Ваше пред 0
Да, пред 1
Да, пред 1
Нет, пред 0
Ваше пред 0
Да, пред 1
Нет, пред 0
Да, пред 1
Да, пред 1
Да, пред 1
Нет, пред 0
Да, пред 1
Нет, пред 0
Да, пред 1
Да, пред 1
Нет, пред 0
Нет, пред 0
Ваше пред 0
Нет, пред 0
Да, пред 1
Ваша послед 0
Ваше пред 0
Да, пр

In [58]:
pd.DataFrame(results, index = ['Accuracy','Precision','Recall','F1-score']).T

,Accuracy,Precision,Recall,F1-score
Qwen2.5-1.5B-Instruct / ru base prompt,0.509664,0.727768,0.547067,0.624611


### 3.2 Yes/No ru prompt

In [60]:
prompt = YESNO_RU_PROMPT
prompt

'Предыдущее предложение корректно ? Отвечай только Да или Нет.'

In [61]:
results[model.name_or_path.split('/')[1] + ' / ru yesno prompt'] = get_model_results(prompt, labels_true)

  0%|          | 0/983 [00:00<?, ?it/s]

Нет.<|im_end|> 0
Нет<|im_end|> 0
Нет.<|im_end|> 0
Нет.<|im_end|> 0
Нет<|im_end|> 0
Нет<|im_end|> 0
Нет.<|im_end|> 0
Нет<|im_end|> 0
Нет<|im_end|> 0
Нет.<|im_end|> 0
Нет<|im_end|> 0
Нет<|im_end|> 0
Нет.<|im_end|> 0
Нет<|im_end|> 0
Нет<|im_end|> 0
Нет<|im_end|> 0
Нет.<|im_end|> 0
Нет<|im_end|> 0
Нет<|im_end|> 0
Нет<|im_end|> 0
Нет<|im_end|> 0
Нет.<|im_end|> 0
Нет.<|im_end|> 0
Нет<|im_end|> 0
Нет.<|im_end|> 0
Нет.<|im_end|> 0
Нет.<|im_end|> 0
Нет<|im_end|> 0
Нет.<|im_end|> 0
Нет.<|im_end|> 0
Нет.<|im_end|> 0
Нет.<|im_end|> 0
Нет.<|im_end|> 0
Нет<|im_end|> 0
Нет.<|im_end|> 0
Нет<|im_end|> 0
Нет.<|im_end|> 0
Нет<|im_end|> 0
Нет<|im_end|> 0
Нет.<|im_end|> 0
Нет.<|im_end|> 0
Нет<|im_end|> 0
Нет.<|im_end|> 0
Нет<|im_end|> 0
Нет.<|im_end|> 0
Нет<|im_end|> 0
Нет<|im_end|> 0
Нет.<|im_end|> 0
Нет.<|im_end|> 0
Нет.<|im_end|> 0
Нет<|im_end|> 0
Нет<|im_end|> 0
Нет<|im_end|> 0
Нет.<|im_end|> 0
Нет.<|im_end|> 0
Нет.<|im_end|> 0
Нет<|im_end|> 0
Нет<|im_end|> 0
Нет.<|im_end|> 0
Нет<|im_end|> 0
Нет.<|im_e

In [62]:
pd.DataFrame(results, index = ['Accuracy','Precision','Recall','F1-score']).T

,Accuracy,Precision,Recall,F1-score
Qwen2.5-1.5B-Instruct / ru base prompt,0.509664,0.727768,0.547067,0.624611
Qwen2.5-1.5B-Instruct / ru yesno prompt,0.261445,0.818182,0.012278,0.024194


### 3.3 Base en prompt

In [64]:
prompt = BASE_EN_PROMPT
prompt

'Is the previous sentence correct? Response translate to russian language.'

In [65]:
results[model.name_or_path.split('/')[1] + ' / en base prompt'] = get_model_results(prompt, labels_true)

  0%|          | 0/983 [00:00<?, ?it/s]

В прошл 0
Ответить на 0
Ответ на р 0
The previous sentence is 0
Конечно, 0
Да, пред 1
Да, пред 1
Да, пред 1
Конечно, 0
The given sentence is 0
Да, пред 1
The previous sentence is 0
Ответить на 0
Конечно, 0
Следующ 0
Конечно, 0
Да, пред 1
Ответ на р 0
The previous sentence is 0
Да, пред 1
Ответить на 0
The previous sentence is 0
Вопрос перев 0
Нет, пред 0
Конечно, 0
Нет, пред 0
Да, пред 1
The previous sentence is 0
Ответить на 0
The previous sentence is 0
Да, пред 1
Конечно, 0
Вот рус 0
Прошлый 0
The previous sentence is 0
Да, пред 1
Ответить на 0
Он не был 0
Верно ли 0
Его вера 0
The previous sentence is 0
Ответить на 0
Следующ 0
Да, пред 1
Ответ на р 0
Вот рус 0
Ответ на ваш 0
Да, пред 1
Да, пред 1
Нет, пред 0
Да, пред 1
Yes, the previous 0
Конечно, 0
The previous sentence is 0
Конечно, 0
The previous sentence is 0
Конечно, 0
Ответить на 0
The previous sentence is 0
Да, пред 1
Слово " 0
Да, пред 1
Да, пред 1
The previous sentence is 0
Конечно, 0
Конечно, 0
Ответить на 0
Следующ 0
Коне

In [66]:
pd.DataFrame(results, index = ['Accuracy','Precision','Recall','F1-score']).T

,Accuracy,Precision,Recall,F1-score
Qwen2.5-1.5B-Instruct / ru base prompt,0.509664,0.727768,0.547067,0.624611
Qwen2.5-1.5B-Instruct / ru yesno prompt,0.261445,0.818182,0.012278,0.024194
Qwen2.5-1.5B-Instruct / en base prompt,0.351984,0.737624,0.203274,0.318717


# 3.4 Yes/No en prompt

In [68]:
prompt = YESNO_EN_PROMPT
prompt

'Is the previous sentence correct? Answer only Yes or No. Response translate to russian language.'

In [69]:
results[model.name_or_path.split('/')[1] + ' / en yesno prompt'] = get_model_results(prompt, labels_true)

  0%|          | 0/983 [00:00<?, ?it/s]

No<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
Нет<|im_end|> 0
Yes<|im_end|> 0
Нет<|im_end|> 0
Yes<|im_end|> 0
No<|im_end|> 0
No<|im_end|> 0
Yes<|im_end|> 0
No<|im_end|> 0
Yes<|im_end|> 0
Нет<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
No<|im_end|> 0
Нет<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
No<|im_end|> 0
Yes<|im_end|> 0
No<|im_end|> 0
No<|im_end|> 0
Yes<|im_end|> 0
Да<|im_end|> 1
Yes<|im_end|> 0
Yes<|im_end|> 0
No<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
No<|im_end|> 0
Нет<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
No<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
No<|im_end|> 0
No<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
No<|im_end|> 0
Yes<|im_end|> 0
No<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
Yes<|im_end|> 0
Нет<|im_end|> 0
Yes<|im_end|> 0
Нет<|im_end|> 0
Yes<|im_end|> 0
No<|im_end|> 0
Yes<|im_end|> 0
No<|im_en

In [70]:
pd.DataFrame(results, index = ['Accuracy','Precision','Recall','F1-score']).T

,Accuracy,Precision,Recall,F1-score
Qwen2.5-1.5B-Instruct / ru base prompt,0.509664,0.727768,0.547067,0.624611
Qwen2.5-1.5B-Instruct / ru yesno prompt,0.261445,0.818182,0.012278,0.024194
Qwen2.5-1.5B-Instruct / en base prompt,0.351984,0.737624,0.203274,0.318717
Qwen2.5-1.5B-Instruct / en yesno prompt,0.261445,0.769231,0.013643,0.026810


## 4. Добавим System Prompt

Будем использовать следующие промпты:

In [138]:
SYSTEM_PROMPT_RU = "Ты Квен, эксперт по лингвистике. Твоя задача проверять является ли текст грамматически, синтаксически и семантически корректным."
SYSTEM_PROMPT_EN = "You are Qwen, a linguistic expert. Your job is to check whether the text is grammatically, syntactically, and semantically correct."

Учитывая печальные результаты с вариантом "только Да/Нет", ограничимся комбинацией базового и системного промпта на русском и английском. 

### 4.1 Base ru prompt + system

In [75]:
prompt = BASE_RU_PROMPT
prompt

'Предыдущее предложение корректно ?'

In [76]:
results[model.name_or_path.split('/')[1] + ' / ru base+sys prompt'] = get_model_results(prompt, labels_true, sys_prompt=SYSTEM_PROMPT_RU )

  0%|          | 0/983 [00:00<?, ?it/s]

Да, пред 1
Да, пред 1
Да, пред 1
Нет, пред 0
Да, пред 1
Да, пред 1
Нет, пред 0
Нет, пред 0
Нет, пред 0
Да, пред 1
Нет, пред 0
Нет, пред 0
Да, пред 1
Да, пред 1
Нет, пред 0
Да, пред 1
Да, пред 1
Нет, пред 0
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Нет, пред 0
Да, пред 1
Да, пред 1
Нет, пред 0
Нет, пред 0
Да, пред 1
Нет, пред 0
Да, пред 1
Нет, пред 0
Да, пред 1
Да, пред 1
Нет, пред 0
Да, пред 1
Да, пред 1
Да, пред 1
Нет, пред 0
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Нет, пред 0
Да, пред 1
Да, пред 1
Нет, пред 0
Нет, пред 0
Нет, пред 0
Нет, пред 0
Да, пред 1
Да, пред 1
Да, пред 1
Нет, пред 0
Да, пред 1
Нет, пред 0
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Да, пред 1
Нет, пред 0
Неточностей 0
Да, пред 1
Нет, пред 0
Да, пред 1
Да, пред 1
Да, пред 1
Нет, пред 0
Да, пред 1
Да, пред 1
Нет, пред 0
Да,

In [77]:
pd.DataFrame(results, index = ['Accuracy','Precision','Recall','F1-score']).T

,Accuracy,Precision,Recall,F1-score
Qwen2.5-1.5B-Instruct / ru base prompt,0.509664,0.727768,0.547067,0.624611
Qwen2.5-1.5B-Instruct / ru yesno prompt,0.261445,0.818182,0.012278,0.024194
Qwen2.5-1.5B-Instruct / en base prompt,0.351984,0.737624,0.203274,0.318717
Qwen2.5-1.5B-Instruct / en yesno prompt,0.261445,0.769231,0.013643,0.026810
Qwen2.5-1.5B-Instruct / ru base+sys prompt,0.589013,0.733333,0.705321,0.719054


### 4.2 Base en prompt + system

In [142]:
prompt = BASE_EN_PROMPT
prompt

'Is the previous sentence correct? Response translate to russian language.'

In [144]:
results[model.name_or_path.split('/')[1] + ' / en base+sys prompt'] = get_model_results(prompt, labels_true, sys_prompt=SYSTEM_PROMPT_EN)

  0%|          | 0/983 [00:00<?, ?it/s]

The sentence "И 0
Yes, the previous 0
Yes, the previous 0
Yes, the previous 0
Yes, the previous 0
Yes, the previous 0
Yes, the previous 0
Yes, the previous 0
Yes, the previous 0
Yes, the sentence 0
Yes, the previous 0
The sentence "А 0
Yes, the previous 0
The sentence "Т 0
Yes, the previous 0
Yes, the sentence 0
Yes, the sentence 0
The given sentence " 0
The sentence "Н 0
The previous sentence is 0
Yes, the sentence 0
Yes, the sentence 0
Да, пред 1
The previous sentence is 0
The translation of your 0
Yes, the previous 0
Да, пред 1
Yes, the previous 0
The previous sentence " 0
The previous sentence is 0
Yes, the previous 0
The translation of your 0
Yes, the sentence 0
The given sentence is 0
Yes, the sentence 0
Yes, the sentence 0
Yes, the previous 0
The sentence "Е 0
The sentence "О 0
Yes, the previous 0
Yes, the previous 0
The translation of " 0
The previous sentence is 0
The given sentence in 0
Yes, the sentence 0
Yes, the sentence 0
Yes, the previous 0
Yes, the previous 0
Yes, the s

In [146]:
pd.DataFrame(results, index = ['Accuracy','Precision','Recall','F1-score']).T

,Accuracy,Precision,Recall,F1-score
Qwen2.5-1.5B-Instruct / ru base prompt,0.509664,0.727768,0.547067,0.624611
Qwen2.5-1.5B-Instruct / ru yesno prompt,0.261445,0.818182,0.012278,0.024194
Qwen2.5-1.5B-Instruct / en base prompt,0.351984,0.737624,0.203274,0.318717
Qwen2.5-1.5B-Instruct / en yesno prompt,0.261445,0.769231,0.013643,0.026810
Qwen2.5-1.5B-Instruct / ru base+sys prompt,0.589013,0.733333,0.705321,0.719054
Qwen2.5-1.5B-Instruct / en base+sys prompt,0.251272,0.363636,0.005457,0.010753


Сочетание промтов на английском и текста на русском путает модель и приводит к смешанным ответам на русском и английском.

## 5. Сравнение результатов

Отсортируем полученные результаты по метрике F1, показывающую сбалансированный результат:

In [150]:
pd.DataFrame(results, index = ['Accuracy','Precision','Recall','F1-score']).T.sort_values(by=['F1-score'], ascending=False)

,Accuracy,Precision,Recall,F1-score
Qwen2.5-1.5B-Instruct / ru base+sys prompt,0.589013,0.733333,0.705321,0.719054
Qwen2.5-1.5B-Instruct / ru base prompt,0.509664,0.727768,0.547067,0.624611
Qwen2.5-1.5B-Instruct / en base prompt,0.351984,0.737624,0.203274,0.318717
Qwen2.5-1.5B-Instruct / en yesno prompt,0.261445,0.769231,0.013643,0.026810
Qwen2.5-1.5B-Instruct / ru yesno prompt,0.261445,0.818182,0.012278,0.024194
Qwen2.5-1.5B-Instruct / en base+sys prompt,0.251272,0.363636,0.005457,0.010753


Полученные результаты:
- Добавление системного промпта на русском к базовому русскому промпту показывает наилучшие результаты;
- Смесь английских промптов с русским текстом задачи путает модель и затрудняет интерпретацию результатов;
- При попытках ограничить ответы модели только Да/Нет модель продемонстрировала худшие результаты на обоих языках; 

Основные выводы:
- Использование системного промта улучшает результаты;
- Для пакетной обработки результатов желательно, чтобы язык промптов совпадал с языком задачи. 